In [1]:
import os
import sys
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
import re
import string
import torch.nn.functional as F

### EDA

### Data preprocessing

In [2]:
nameDict = {0:'id', 1:'claim_author', 2:'claim', 3:'tweet', 4:'screening', 5:'answered', 6:'tweet_url_title', 7:'claim_timestamp', 8:'tweet_timestamp',
            9:'tweet_id', 10:'tweet_userhandle', 11:'retweet_count', 12:'reply_count', 13:'like_count', 14:'quote_count', 15:'claim_source',
            16:'claim_verdict', 17:'factcheck_timestamp', 18:'claim_review_summary', 19:'claim_review', 20:'factcheck_url', 21:'claim_tags',
            22:'claimbuster_score', 23:'pair_id', 24:'factcheck_claim_author_url', 25:'claim_timestamp_formatted', 26:'claim_author_description',
            27:'subset', 28:'annotation_score'}

In [31]:
pd.set_option('display.max_columns', None)
df = pd.read_json("./Model/Datasets/annotator_agreements.json")
df = df.rename(columns=nameDict)
df.head()

id      claim_author                                              claim  \
0  211   Instagram posts  "Special Forces arrest Deep State Dr. Anthony ...   
1  252            Tweets  "With the money Elon Musk spent to buy Twitter...   
2  262    Facebook posts  "Colorado Governor Jared Polis has just signed...   
3  281  Donald Trump Jr.                The FBI lost Hunter Biden's laptop.   
4  291  Donald Trump Jr.                The FBI lost Hunter Biden's laptop.   

                                               tweet  screening  answered  \
0  Please let this be true. https://trusttheq.com...         -3         4   
1  With the money Elon Musk spent to buy Twitter ...         -3         3   
2  Colorado Governor Jared Polis has just signed ...         -3         5   
3  So are the rumors true? The FBI lost Hunter Bi...         -3         5   
4  Hunter's Lost Laptop: FBI trends over alleged ...         -3         4   

                                     tweet_url_title claim_timestamp  \
0  https://trusttheq.com/special-forces-arrest-de...      2022-04-11   
1                                                         2022-04-05   
2  https://coloradosun.com/2022/04/04/colorado-ab...      2022-04-05   
3                                                         2022-04-04   
4  https://www.herald.ng/fbi-trends-over-hunters-...      2022-04-04   

            tweet_timestamp             tweet_id     tweet_userhandle  \
0  2022-04-11T14:49:34.000Z  1513529698940329984             25125328   
1  2022-04-06T15:23:07.000Z  1511726202410524672             95257097   
2  2022-04-07T14:50:45.000Z  1512080444539375616  1511668016261615616   
3  2022-05-01T00:56:02.000Z  1520567688224985088  1399213487553929216   
4  2022-04-02T09:47:58.000Z  1510192307856891904            941320202   

   retweet_count  reply_count  like_count  quote_count claim_source  \
0              1            1           3            0   politifact   
1            327         1740        5005          164   politifact   
2              0            0           0            0   politifact   
3              0            0           0            0   politifact   
4              0            0           0            0   politifact   

  claim_verdict factcheck_timestamp  \
0    pants-fire          2022-04-12   
1    pants-fire          2022-04-08   
2         false          2022-04-07   
3         false          2022-04-06   
4         false          2022-04-06   

                                claim_review_summary  \
0  No, Dr. Anthony Fauci was not arrested.   Dr. ...   
1  Twitter post wasn't serious about Elon Musk us...   
2  No, this Colorado law does not allow abortions...   
3  The FBI lost Hunter Biden's laptop? There's no...   
4  The FBI lost Hunter Biden's laptop? There's no...   

                                        claim_review  \
0  Following other false claims that Dr. Anthony ...   
1  A viral social media post is claiming the $3 b...   
2  While state lawmakers in Oklahoma and Texas ar...   
3  For well over a year, we've heard a lot about ...   
4  For well over a year, we've heard a lot about ...   

                                       factcheck_url  \
0  https://www.politifact.com/factchecks/2022/apr...   
1  https://www.politifact.com/factchecks/2022/apr...   
2  https://www.politifact.com/factchecks/2022/apr...   
3  https://www.politifact.com/factchecks/2022/apr...   
4  https://www.politifact.com/factchecks/2022/apr...   

                                         claim_tags  claimbuster_score  \
0             Facebook Fact-checks, Instagram posts                NaN   
1    Debt, Population, Facebook Fact-checks, Tweets                NaN   
2                Abortion, Colorado, Facebook posts                NaN   
3  National, Facebook Fact-checks, Donald Trump Jr.                NaN   
4  National, Facebook Fact-checks, Donald Trump Jr.                NaN   

                                    pair_id  \
0  211-5c1df5a9-2a70-4988-99e

In [32]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
    
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

df['tweet']=df['tweet'].apply(lambda x : remove_URL(x))
df['tweet']=df['tweet'].apply(lambda x : remove_punct(x))

In [33]:
# df['text'] = (df['claim'].map(str) +' '+ df['tweet']).apply(lambda row: row.strip())
df = df.rename(columns={'claim':'sentence1','tweet':'sentence2','annotation_score':'labels'})
df

id      claim_author  \
0       211   Instagram posts   
1       252            Tweets   
2       262    Facebook posts   
3       281  Donald Trump Jr.   
4       291  Donald Trump Jr.   
...     ...               ...   
1451  33440      Wayne Powell   
1452  34580     Newt Gingrich   
1453  35381      Donald Trump   
1454  35400  Michele Bachmann   
1455  35981      Robert Gates   

                                              sentence1  \
0     "Special Forces arrest Deep State Dr. Anthony ...   
1     "With the money Elon Musk spent to buy Twitter...   
2     "Colorado Governor Jared Polis has just signed...   
3                   The FBI lost Hunter Biden's laptop.   
4                   The FBI lost Hunter Biden's laptop.   
...                                                 ...   
1451  Eric Cantor "took $5 million from Sheldon Adel...   
1452  Students today "take more years to get through...   
1453  "CNN did a poll recently where Obama and I are...   
1454  "The top 1 percent of income-earners pay about...   
1455  "The video doesn't show the broader picture of...   

                                              sentence2  screening  answered  \
0                              Please let this be true          -3         4   
1     With the money Elon Musk spent to buy Twitter ...         -3         3   
2     Colorado Governor Jared Polis has just signed ...         -3         5   
3     So are the rumors true The FBI lost Hunter Bid...         -3         5   
4     Hunters Lost Laptop FBI trends over alleged co...         -3         4   
...                                                 ...        ...       ...   
1451  Wayne Powell Eric Cantor took 5 million from S...          0         3   
1452  Gingrich says students today take more years t...         -1         3   
1453  Donald Trump CNN did a poll and it shows im ti...          0         5   
1454  FALSE Michelle Bachmann The top 1 percent of i...         -1         3   
1455  Gates told ABCs This Week that the video quote...          0         6   

                                        tweet_url_title claim_timestamp  \
0     https://trusttheq.com/special-forces-arrest-de...      2022-04-11   
1                                                            2022-04-05   
2     https://coloradosun.com/2022/04/04/colorado-ab...      2022-04-05   
3                                                            2022-04-04   
4     https://www.herald.ng/fbi-trends-over-hunters-...      2022-04-04   
...                                                 ...             ...   
1451                                                         2012-10-01   
1452                                                         2012-01-28   
1453                                                         2011-04-27   
1454                                                         2011-04-13   
1455                                                         2010-04-11   

               tweet_timestamp             tweet_id     tweet_userhandle  \
0     2022-04-11T14:49:34.000Z  1513529698940329984             25125328   
1     2022-04-06T15:23:07.000Z  1511726202410524672             95257097   
2     2022-04-07T14:50:45.000Z  1512080444539375616  1511668016261615616   
3     2022-05-01T00:56:02.000Z  1520567688224985088  1399213487553929216   
4     2022-04-02T09:47:58.000Z  1510192307856891904            941320202   
...                        ...                  ...                  ...   
1451  2012-10-15T19:18:01.000Z   257923301849059328            419328966   
1452  2012-02-17T00:25:04.000Z   170302707133988864             67095221   
1453  2011-04-27T14:17:52.000Z    63245482402660352             17933641   
1454  2011-04-22T11:40:49.000Z    61394018755559424             67095221   
1455  2010-04-12T18:30:23.000Z          12059638618             24789484   

      retweet_count  reply_count  like_count  quote_count claim_source  \
0                 1            1           3          

In [35]:
df['labels'] = df['labels']+1
df = df.loc[:, ['sentence1', 'sentence2','labels']]
dataset = Dataset(pa.Table.from_pandas(df))
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'labels'],
    num_rows: 1456
})

In [36]:
df['labels'].unique()

array([1, 2, 3, 0], dtype=int64)

In [37]:
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'labels'],
        num_rows: 1164
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'labels'],
        num_rows: 292
    })
})

### Model

In [84]:
import numpy as np
import evaluate
from datasets import Dataset, load_dataset
from transformers import DataCollatorWithPadding, BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

In [85]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  weight_decay=0.01)
data_collator = DataCollatorWithPadding(tokenizer)
metric = evaluate.load('accuracy')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], padding='max_length', truncation=True)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [87]:
tokenized_ds = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1164 [00:00<?, ? examples/s]

Map:   0%|          | 0/292 [00:00<?, ? examples/s]

In [88]:
small_train = tokenized_ds['train'].shuffle(seed=30).select(range(750))

In [89]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=tokenized_ds['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [90]:
trainer.train()

TrainOutput(global_step=188, training_loss=1.138869508783868, metrics={'train_runtime': 2790.3365, 'train_samples_per_second': 0.538, 'train_steps_per_second': 0.067, 'total_flos': 394673670144000.0, 'train_loss': 1.138869508783868, 'epoch': 2.0})

### Prediction and Stat

In [91]:
predictions = trainer.predict(tokenized_ds['test'])
print(predictions.predictions.shape, predictions.label_ids.shape)

(292, 4) (292,)


In [92]:
preds = np.argmax(prediction.predictions, axis=-1)
metrics.compute(predictions=preds, references=prediction.label_ids)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

#### Practic ground

In [ ]:
encoding = tokenizer([[df['tweet'][0], df['claim'][0]]], truncation=True, padding=True)
with torch.no_grad():
    encoding['labels'] = [0]
    test_tensor = {k: torch.tensor(v).to(device) for k, v in encoding.items()}
    outputs = model(**test_tensor)
    logits = outputs.logits
    probs = F.softmax(logits, dim=1).tolist()[0]


In [35]:
probs

[0.321289986371994, 0.205239400267601, 0.2673124372959137, 0.20615814626216888]